In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.datasets import make_classification

In [3]:
from sklearn.model_selection import train_test_split

In [5]:
x,y=make_classification(n_samples=2000,n_classes=2,weights=[1,1],random_state=1)

In [6]:
x.shape

(2000, 20)

In [8]:
y.shape

(2000,)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

In [10]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

##rendom forest

## rendom forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rf_model = RandomForestClassifier()

In [13]:
rf_model.fit(x_train,y_train)

RandomForestClassifier()

In [14]:
ytrain_pred = rf_model.predict_proba(x_train)

In [15]:
print('RF train roc-auc : {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))

RF train roc-auc : 1.0


In [17]:
ytest_pred=rf_model.predict_proba(x_test)

In [18]:
print('PF test roc-auc : {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))

PF test roc-auc : 0.9821666666666667


In [19]:
ytrain_pred

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.01, 0.99],
       ...,
       [0.98, 0.02],
       [1.  , 0.  ],
       [0.23, 0.77]])

## logistic prediction

In [22]:
from sklearn.linear_model import LogisticRegression
log_classifier=LogisticRegression()
log_classifier.fit(x_train,y_train)
ytrain_pred = log_classifier.predict_proba(x_train)
print('logistic tarin roc-auc : {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))

logistic tarin roc-auc : 0.9863568922694498


In [23]:
ytest_pred = log_classifier.predict_proba(x_test)
print('logistic test roc-auc : {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))

logistic test roc-auc : 0.9885777777777777


## adaboost

In [24]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier = AdaBoostClassifier()
ada_classifier.fit(x_train,y_train)
ytrain_pred = ada_classifier.predict_proba(x_train)
print('adaboost train roc-auc : {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))

adaboost train roc-auc : 0.9975081174960356


In [25]:
yteat_pred = ada_classifier.predict_proba(x_test)
print('adaboost test roc-auc : {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))

adaboost test roc-auc : 0.9885777777777777


## knn classifier

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(x_train,y_train)
ytrain_pred = knn_classifier.predict_proba(x_train)
print('knn train roc-auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))

knn train roc-auc: 0.981670071491109


In [28]:
ytest_pred = knn_classifier.predict_proba(x_test)
print('knn test : {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))

knn test : 0.9426111111111111


### now we will focus on the best threshold for maximum accuracy

In [31]:
pred=[]
for model in [rf_model,log_classifier,ada_classifier,knn_classifier]:
    pred.append(pd.Series(model.predict_proba(x_test)[:,1]))
    
final_prediction = pd.concat(pred,axis=1).mean(axis=1)
print('ensemble test roc-auc : {}'.format(roc_auc_score(y_test,final_prediction)))

ensemble test roc-auc : 0.9845222222222222


In [32]:
pd.concat(pred,axis=1)

,0,1,2,3
0,0.99,0.991861,0.559186,1.0
1,0.01,0.000008,0.463282,0.0
2,0.98,0.966929,0.538202,0.8
3,0.94,0.761539,0.509875,0.8
4,0.65,0.779443,0.490344,0.4
...,...,...,...,...
595,0.00,0.024239,0.461121,0.0
596,0.03,0.000003,0.441377,0.0
597,0.99,0.984385,0.532403,1.0
598,0.02,0.001147,0.441720,0.2


In [33]:
final_prediction

0      0.885262
1      0.118323
2      0.821283
3      0.752853
4      0.579947
         ...   
595    0.121340
596    0.117845
597    0.876697
598    0.165717
599    0.837357
Length: 600, dtype: float64

In [34]:
fpr, tpr, thresholds = roc_curve(y_test,final_prediction)
thresholds

array([1.91188114, 0.91188114, 0.90728779, 0.90577475, 0.80634271,
       0.8055558 , 0.80540031, 0.80412833, 0.78390513, 0.78321156,
       0.76887064, 0.76787124, 0.73930851, 0.73887362, 0.69294088,
       0.69143711, 0.66601868, 0.65493537, 0.62207293, 0.62175354,
       0.5990152 , 0.59864346, 0.59212652, 0.59050386, 0.56595371,
       0.56236186, 0.55355932, 0.54889283, 0.53865858, 0.51366892,
       0.46490225, 0.46294916, 0.4471034 , 0.38905357, 0.38129719,
       0.35245613, 0.35086612, 0.23698424, 0.23646893, 0.2294954 ,
       0.22890421, 0.19789681, 0.19598417, 0.11784501, 0.1178351 ,
       0.10498954])

In [35]:
from sklearn.metrics import accuracy_score
accuracy_ls = []
for thres in thresholds:
    y_pred = np.where(final_prediction>thres,1.0)
    accuracy_ls.append(accuracy_score(y_test,y_pred,normalize=True))
    
accuracy_ls = pd.concat([pd.Series(thresholds),pd.Series(accuracy_ls)],axis=1)
accuracy_ls_columns = ['thresholds','accuracy']
acuracy_ls.sort_values(by='accuracy',ascending=False,inplace=True)
accuracy_ls.head()

ValueError: either both or neither of x and y should be given